# SuperKamiokande event 3D displayer + animation

<img src="figure/superk_page.png">
**SuperKamiokande**, ICRR has released their simulated data for plotting as below link.
This tutorial shows how to use Plotly to draw interactive graphs.

http://www-sk.icrr.u-tokyo.ac.jp/sk/detector/eventdisplay-data.html?fbclid=IwAR3XvOjN1BbrgbPzzWPxVKXy4DX0UXSx6BxNYJx-QYuPVHdDSCVHFYZvN7Q

The interactive plots is made by using **Plotly** (website:https://plotly.com/python/) 


A muon sample draw by plotly is shown as below:

<img src="figure/muon_ring.png">

Data lience belong to ICRR, Tokyo Univ.


(website: http://www-sk.icrr.u-tokyo.ac.jp/sk/index.html)

In [ ]:
# install pypi packages in requirements.txt (if needed)
#!pip3 install -r requirements.txt

In [ ]:
# import packages
import os
import numpy as np
import pandas as pd
from zipfile import ZipFile
import glob

# plotly importing with embedding in joupter notebook 
from plotly.offline import init_notebook_mode,iplot
#import plotly.graph_objs as go # old version of plotly.graph_objects (plotly version 3.10.0)
import plotly.graph_objects as go

#import cufflinks as cf
init_notebook_mode(connected=True)

In [ ]:
## download data zip file from icrr website
!wget -O data/sample.zip http://www-sk.icrr.u-tokyo.ac.jp/sk/evdisplay-data/sample.zip
#!wget -O data/1ring-mu.100_events.zip http://www-sk.icrr.u-tokyo.ac.jp/sk/evdisplay-data/1ring-mu.100_events.zip
#!wget -O data/1ring-e.100_events.zip http://www-sk.icrr.u-tokyo.ac.jp/sk/evdisplay-data/1ring-e.100_events.zip
#!wget -O data/multirings.100_events.zip http://www-sk.icrr.u-tokyo.ac.jp/sk/evdisplay-data/multirings.100_events.zip

In [ ]:
fpd={}
names = ['cable', 'charge', 'time', 'x', 'y', 'z']
with ZipFile('data/sample.zip') as zf:
    allfiles = zf.namelist()
    
    sample_names = [file for file in allfiles if '.csv' in file if 'sample/' in file if '__MACOS' not in file]
    #for sname in sample_names:
    #    zf.extract(sname, path='skdata/')
    #csvfiles = glob.glob('skdata/sample/*.csv')
    #print(csvfiles)
    #for filename in csvfiles:
    for filename in sample_names:
        print('readin file {}'.format(filename))
        key = os.path.split(filename)[-1] [:-4]# remove path and '.csv' in the tail, then use it as key
        fpd[key]=pd.read_csv(zf.open(filename),comment='#', names=names)

print(sample_names)

In [ ]:
cat_list = [key for key in fpd.keys()]
print(cat_list)

In [ ]:
# print out head of data
fpd[cat_list[0]].head()

In [ ]:
#preparing dropdown list menu
#defining list_updatemenus, data list
list_updatemenus = []
data = []
for i,cat in enumerate(cat_list):
    item_menu = dict(label=cat,method='update')
    # args list
    args_list = list()
    # preparing visible list
    visible_list = [False]*len(cat_list)
    visible_list[i] = True
    args_list.append(dict(visible=visible_list))
    # preparing title
    title_dict = dict(title='SuperK {} Event'.format(cat))
    args_list.append(title_dict)
    
    item_menu['args'] = args_list
    list_updatemenus.append(item_menu)
    # preparing 3d-scattering plot
    # text for showing
    text_tc = fpd[cat]['time'].map('Time:{:,.1f} ns\t'.format) + fpd[cat]['charge'].map('Charge:{:,.1f} p.e.'.format)
    data.append(go.Scatter3d(x=fpd[cat].x,y=fpd[cat].y,z=fpd[cat].z,mode='markers',text=text_tc,
                marker=dict(size=3,color=fpd[cat].charge,colorscale='Portland',opacity=0.4,
                            colorbar=dict(title="Charge(p.e.)",thickness=10,x = 0.92, y = 0.5))))
    
# print(list_updatemenus[1])

Plotly color built-in pallet 
https://plotly.com/python/builtin-colorscales/

colorscale options for colorbar: ['Greys', 'YlGnBu', 'Greens', 'YlOrRd', 'Bluered', 'RdBu','Reds', 'Blues', 'Picnic', 'Rainbow', 'Portland', 'Jet','Hot', 'Blackbody', 'Earth', 'Electric', 'Viridis', 'Cividis']

In [ ]:
import plotly.graph_objs.layout.scene as scene
#defining layout
layout=go.Layout(title='SuperK Event 3D Scatter Plot',
                 updatemenus=list([dict(buttons= list_updatemenus)]),barmode='overlay',
                 scene=go.Scene(
                     xaxis=scene.XAxis(title='x(cm)'),yaxis=scene.YAxis(title='y(cm)'),zaxis=scene.ZAxis(title='z(cm)'))
                )

In [ ]:
#defining figure and plotting
fig = go.Figure(data,layout)
iplot(fig)

### Animation drawing 
Showing the charge by varing time 
However, the scatter3d type did not suupport animate, so only 2d plot is shown.


Example of animate by plotly:

https://plotly.com/python/animations/#adding-control-buttons-to-animations

In [ ]:
multiring_cat = 'multirings.0000.000014'
datadf = fpd[multiring_cat]
min_time = datadf['time'].min()
max_time = datadf['time'].max()
#print(min_time,max_time)
stamp_first = 50*(min_time//50+1)
stamp_last = 50*(max_time//50+1)
times_list = np.linspace(stamp_first,stamp_last,int((stamp_last-stamp_first)/50+1))
times_list=times_list.astype(int)
#print(times_list)

In [ ]:
# group data with time by 100 ns
# create a new column (timehns): replace time with 100 ns for example 132 ns-->200ns
datadf['timehns'] = ((datadf['time']//50+1)*50).astype(int)
datadf.tail()

In [ ]:
# make figure
fig_dict = {
    "data":[],
    "layout":{},
    "frames":[]
}


# prepare initial data
data_by_time = datadf[datadf['timehns']==times_list[0]]
text_tc = data_by_time['z'].map('Z(cm):{:,.1f}'.format) + data_by_time['charge'].map('\tCharge:{:,.1f} p.e.'.format)

data_dict = {
            "type":"scatter",
            "x":list(data_by_time["x"]),
            "y":list(data_by_time["y"]),
            #"z":list(data_by_time["z"]),
            "mode":"markers",
            "text":text_tc,
            "marker":dict(size=3,color=list(data_by_time.charge),colorscale='Portland',opacity=0.4,
                            colorbar=dict(title="Charge(p.e.)",thickness=10,x = 1.02, y = 0.5)),
            "name": str(times_list[0])
    }
fig_dict["data"].append(data_dict)
    

# prepare layout 
fig_dict["layout"]["xaxis"] = {"title": "x (cm)"}
fig_dict["layout"]["yaxis"] = {"title": "y (cm)"}
#fig_dict["layout"]["zaxis"] = {"title": "z (cm)"}
fig_dict["layout"]["width"] = 600
fig_dict["layout"]["height"] = 600

fig_dict["layout"]["scene"]=go.Scene(xaxis=scene.XAxis(title='x(cm)',range=[-1600, 1600], autorange=False),
                                     yaxis=scene.YAxis(title='y(cm)',range=[-1600, 1600]))

fig_dict["layout"]["xaxis"] = go.layout.XAxis(title='x(cm)',range=[-1600, 1600], autorange=False)           
fig_dict["layout"]["yaxis"] = go.layout.YAxis(title='y(cm)',range=[-1700, 1700], autorange=False)           
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 1500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Time(ns):",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# prepare frames

for time_stamp in times_list:
    frame = {"data": [], "name": str(time_stamp)}
    data_by_time = datadf[datadf['timehns']==time_stamp]
    #print(data_by_time.shape)
    text_tc = data_by_time['z'].map('Z(cm):{:,.1f}'.format) + data_by_time['charge'].map('\tCharge:{:,.1f} p.e.'.format)
    data_dict = {
                "type":"scatter",
                "x":list(data_by_time["x"]),
                "y":list(data_by_time["y"]),
                #"z":list(data_by_time["z"]),
                "mode":"markers",
                "text":text_tc,
                "marker":dict(size=3,color=list(data_by_time["charge"]),colorscale='Portland',opacity=0.4,
                                colorbar=dict(title="Charge(p.e.)",thickness=10,x = 1.02, y = 0.5)),
                "name": str(data_by_time)
        }
    frame["data"].append(data_dict)
    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [str(time_stamp)], # first argument should be string 
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": str(time_stamp),
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)

    #print("append data tme:{}".format(time_stamp))

In [ ]:
fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)
fig.show()

In [ ]:
# Then, draw y-z plots
# make figure
fig_dict = {
    "data":[],
    "layout":{},
    "frames":[]
}


# prepare initial data
data_by_time = datadf[datadf['timehns']==times_list[0]]
text_tc = data_by_time['x'].map('X(cm):{:,.1f}'.format) + data_by_time['charge'].map('\tCharge:{:,.1f} p.e.'.format)

data_dict = {
            "type":"scatter",
            "x":list(data_by_time["y"]),
            "y":list(data_by_time["x"]),
            #"z":list(data_by_time["z"]),
            "mode":"markers",
            "text":text_tc,
            "marker":dict(size=3,color=list(data_by_time.charge),colorscale='Portland',opacity=0.4,
                            colorbar=dict(title="Charge(p.e.)",thickness=10,x = 1.02, y = 0.5)),
            "name": str(times_list[0])
    }
fig_dict["data"].append(data_dict)
    

# prepare layout 
fig_dict["layout"]["xaxis"] = {"title": "y (cm)"}
fig_dict["layout"]["yaxis"] = {"title": "z (cm)"}
#fig_dict["layout"]["zaxis"] = {"title": "z (cm)"}
fig_dict["layout"]["width"] = 600
fig_dict["layout"]["height"] = 600


fig_dict["layout"]["xaxis"] = go.layout.XAxis(title='y(cm)',range=[-1600, 1600], autorange=False)           
fig_dict["layout"]["yaxis"] = go.layout.YAxis(title='z(cm)',range=[-1700, 1700], autorange=False)           
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 1500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Time(ns):",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# prepare frames

for time_stamp in times_list:
    frame = {"data": [], "name": str(time_stamp)}
    data_by_time = datadf[datadf['timehns']==time_stamp]
    #print(data_by_time.shape)
    text_tc = data_by_time['x'].map('X(cm):{:,.1f}'.format) + data_by_time['charge'].map('\tCharge:{:,.1f} p.e.'.format)
    data_dict = {
                "type":"scatter",
                "x":list(data_by_time["y"]),
                "y":list(data_by_time["z"]),
                #"z":list(data_by_time["z"]),
                "mode":"markers",
                "text":text_tc,
                "marker":dict(size=3,color=list(data_by_time["charge"]),colorscale='Portland',opacity=0.4,
                                colorbar=dict(title="Charge(p.e.)",thickness=10,x = 1.02, y = 0.5)),
                "name": str(data_by_time)
        }
    frame["data"].append(data_dict)
    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [str(time_stamp)], # first argument should be string 
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": str(time_stamp),
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)

    #print("append data tme:{}".format(time_stamp))

In [ ]:
fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)
fig.show()

### plotly.express animation

In [ ]:
import plotly.express as px

In [ ]:
# Use plotly.express to draw animation here
# first, we have to order the time in series
datadf.sort_values(by=['timehns'],ascending=True,inplace=True)
datadf.head()

In [ ]:
# Draw the animation here
figure=px.scatter(datadf, x="...", y="...", animation_frame="...", 
           color="....")

figure.show()
